In [1]:
import sqlite3 as sql
import time
from shutil import copyfile

### Producing a database for storing SPARQL queries and results

In [2]:
db = 'db_stores/sparql_queries.db'   ### 'data/database.db'

In [3]:
### Connecting to a SQLite database creates the database if it does not already exist
# in the specified folder

# connect and test connection
cn = sql.connect(db)
c = cn.cursor()
c.execute("SELECT datetime('now', 'localtime')")
print(c.fetchone())
cn.close()

('2022-12-26 00:13:56',)


In [10]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# STOP # c.execute('''DROP TABLE query;''')

c.execute('''CREATE TABLE query (pk_query INTEGER PRIMARY KEY,
label VARCHAR (100),
description TEXT,
project VARCHAR (150),
sparql_endpoint VARCHAR (250),
query TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
cn.commit()
cn.close()

In [11]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# STOP # c.execute('''DROP TABLE result;''')

c.execute('''CREATE TABLE result (pk_result INTEGER PRIMARY KEY,
fk_query INTEGER REFERENCES "query" (pk_query) MATCH SIMPLE,
description TEXT,
result TEXT,
insert_data_into TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
cn.commit()
cn.close()

In [12]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'entity'
# STOP #  c.execute('''DROP TABLE entity''')

c.execute('''CREATE TABLE entity
             (pk_entity INTEGER PRIMARY KEY, local_authority BOOLEAN DEFAULT TRUE, 
             fk_same_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, entity_class TEXT, source TEXT,
             fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             entity_std_name TEXT, notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, entity_class, source) ON CONFLICT IGNORE)''')

# Save (commit) the changes 
cn.commit()
cn.close()

In [13]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'property'
# STOP # c.execute('''DROP TABLE property''')

c.execute('''CREATE TABLE property
             (pk_property INTEGER PRIMARY KEY, 
             fk_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, property TEXT, value TEXT, 
             source TEXT, quality INTEGER, fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             additional_property_1 TEXT, value_1 TEXT,
             additional_property_2 TEXT, value_2 TEXT,
             additional_property_3 TEXT, value_3 TEXT,
             notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, property, value, source) ON CONFLICT IGNORE)''')

# Save (commit) the changes 
cn.commit()
cn.close()

### Manually versioning the database

This operation creates a copy of the database identified by a _timestamp_. To be used before any significant operation on the database (modification of the structure, imports by Python scripts, etc.) in order to avoid data loss.

This operation is performed on the data and is different from versioning the code with GIT.

In [22]:
###  Function retrieving and formatting time at runtime
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now
timestamp_formatted_for_file_name()

'20210512_222524'

In [24]:
timestamped_db_copy = db.replace('.db', '_' + timestamp_formatted_for_file_name() + '.sqlite')
copied_db = copyfile(db, timestamped_db_copy)
copied_db

'db_stores/sparql_queries_20210512_222600.sqlite'